In [34]:
from bokeh.io import show, output_notebook, push_notebook
from bokeh.plotting import figure
from bokeh.models import HoverTool, TapTool, DatetimeTickFormatter, ColumnDataSource, DateFormatter
import pandas as pd
from math import radians, degrees
from datetime import datetime
from datetime import time
from bokeh.resources import CDN
from bokeh.embed import file_html

output_notebook()

Loading BokehJS ...

In [2]:
# https://michelanders.blogspot.com/2010/12/calulating-sunrise-and-sunset-in-python.html
from math import cos, sin, acos, asin, tan
from math import degrees as deg, radians as rad
from datetime import date, datetime, time, tzinfo, timedelta
import time as _time


ZERO = timedelta(0)
HOUR = timedelta(hours=1)
SECOND = timedelta(seconds=1)

STDOFFSET = timedelta(seconds = -_time.timezone)
if _time.daylight:
    DSTOFFSET = timedelta(seconds = -_time.altzone)
else:
    DSTOFFSET = STDOFFSET

DSTDIFF = DSTOFFSET - STDOFFSET

class LocalTimezone(tzinfo):

    def fromutc(self, dt):
        assert dt.tzinfo is self
        stamp = (dt - datetime(1970, 1, 1, tzinfo=self)) // SECOND
        args = _time.localtime(stamp)[:6]
        dst_diff = DSTDIFF // SECOND
        # Detect fold
        fold = (args == _time.localtime(stamp - dst_diff))
        return datetime(*args, microsecond=dt.microsecond,
                        tzinfo=self, fold=fold)

    def utcoffset(self, dt):
        if self._isdst(dt):
            return DSTOFFSET
        else:
            return STDOFFSET

    def dst(self, dt):
        if self._isdst(dt):
            return DSTDIFF
        else:
            return ZERO

    def tzname(self, dt):
        return _time.tzname[self._isdst(dt)]

    def _isdst(self, dt):
        tt = (dt.year, dt.month, dt.day,
              dt.hour, dt.minute, dt.second,
              dt.weekday(), 0, 0)
        stamp = _time.mktime(tt)
        tt = _time.localtime(stamp)
        return tt.tm_isdst > 0

class sun:
    """
    Calculate sunrise and sunset based on equations from NOAA
    http://www.srrb.noaa.gov/highlights/sunrise/calcdetails.html

    typical use, calculating the sunrise at the present day:

    import datetime
    import sunrise
    s = sun(lat=49,long=3)
    print('sunrise at ',s.sunrise(when=datetime.datetime.now())
    """

    def __init__(self, lat=52.37, long=4.90):  # default Amsterdam
        self.lat = lat
        self.long = long

    def sunrise(self, when=None):
        """
        return the time of sunrise as a datetime.time object
        when is a datetime.datetime object. If none is given
        a local time zone is assumed (including daylight saving
        if present)
        """
        if when is None:
            when = datetime.now(tz=LocalTimezone())
        self.__preptime(when)
        self.__calc()
        return sun.__timefromdecimalday(self.sunrise_t)

    def sunset(self, when=None):
        if when is None:
            when = datetime.now(tz=LocalTimezone())
        self.__preptime(when)
        self.__calc()
        return sun.__timefromdecimalday(self.sunset_t)

    def solarnoon(self, when=None):
        if when is None:
            when = datetime.now(tz=LocalTimezone())
        self.__preptime(when)
        self.__calc()
        return sun.__timefromdecimalday(self.solarnoon_t)

    @staticmethod

    def __timefromdecimalday(day):
        """
        return a datetime.time object.
        day is a decimal day between 0.0 and 1.0, e.g. noon = 0.5
        """
        hours = 24.0*day
        h = int(hours)
        minutes = (hours-h)*60
        m = int(minutes)
        seconds = (minutes-m)*60
        s = int(seconds)
        return time(hour=h, minute=m, second=s)

    def __preptime(self, when):
        """
        Extract information in a suitable format from when,
        a datetime.datetime object.
        """
        # datetime days are numbered in the Gregorian calendar
        # while the calculations from NOAA are distibuted as
        # OpenOffice spreadsheets with days numbered from
        # 1/1/1900. The difference are those numbers taken for
        # 18/12/2010
        self.day = when.toordinal()-(734124-40529)
        t = when.time()
        self.time = (t.hour + t.minute/60.0 + t.second/3600.0)/24.0

        self.timezone = 0
        offset = when.utcoffset()
        if not offset is None:
            self.timezone = offset.seconds/3600.0

    def __calc(self):
        """
        Perform the actual calculations for sunrise, sunset and
        a number of related quantities.

        The results are stored in the instance variables
        sunrise_t, sunset_t and solarnoon_t
        """ 
        timezone = self.timezone  # in hours, east is positive
        longitude = self.long     # in decimal degrees, east is positive
        latitude = self.lat       # in decimal degrees, north is positive

        time = self.time  # percentage past midnight, i.e. noon  is 0.5
        day = self.day     # daynumber 1=1/1/1900

        Jday = day+2415018.5+time-timezone/24  # Julian day
        Jcent = (Jday-2451545)/36525    # Julian century

        Manom = 357.52911+Jcent*(35999.05029-0.0001537*Jcent)
        Mlong = 280.46646+Jcent*(36000.76983+Jcent*0.0003032) % 360
        Eccent = 0.016708634-Jcent*(0.000042037+0.0001537*Jcent)
        Mobliq = 23+(26+((21.448-Jcent*(46.815+Jcent*(0.00059-Jcent*0.001813))))/60)/60
        obliq = Mobliq+0.00256*cos(rad(125.04-1934.136*Jcent))
        vary = tan(rad(obliq/2))*tan(rad(obliq/2))
        Seqcent = sin(rad(Manom))*(1.914602-Jcent*(0.004817+0.000014*Jcent))+sin(rad(2*Manom))*(0.019993-0.000101*Jcent)+sin(rad(3*Manom))*0.000289
        Struelong = Mlong+Seqcent
        Sapplong = Struelong-0.00569-0.00478*sin(rad(125.04-1934.136*Jcent))
        declination = deg(asin(sin(rad(obliq))*sin(rad(Sapplong))))

        eqtime = 4*deg(vary*sin(2*rad(Mlong))-2*Eccent*sin(rad(Manom))+4*Eccent*vary*sin(rad(Manom))*cos(2*rad(Mlong))-0.5*vary*vary*sin(4*rad(Mlong))-1.25*Eccent*Eccent*sin(2*rad(Manom)))

        hourangle = deg(acos(cos(rad(90.833))/(cos(rad(latitude))*cos(rad(declination)))-tan(rad(latitude))*tan(rad(declination))))

        self.solarnoon_t = (720-4*longitude-eqtime+timezone*60)/1440
        self.sunrise_t = self.solarnoon_t-hourangle*4/1440
        self.sunset_t = self.solarnoon_t+hourangle*4/1440

#if __name__ == "__main__":
#s=sun(lat=52.37,long=4.90)
#print(datetime.today())
#print(s.sunrise(),s.solarnoon(),s.sunset())


In [3]:
file = 'SSD_IMGIinnsbruck.txt'  # http://ertel2.uibk.ac.at:8080/uploads-imgi/2012/03/SSD_IMGIinnsbruck.txt
colnames = ['MM', 'DD', 'SU', 'SA', 'SSD']
year = 2020
dateparser = lambda x: pd.datetime.strptime(x, '2020 %m %d')
df = pd.read_csv(file, 
                 header=21, 
                 delim_whitespace=True, 
                 names=colnames, 
                 #parse_dates={'Date': ['MM', 'DD']}, 
                 #keep_date_col=True,
                 index_col=False,
                 #date_parser=dateparser,
                )
df = df.set_index(pd.to_datetime({'year': year, 'month': df['MM'], 'day': df['DD']}))
df = df.drop(columns=['MM', 'DD'])

# converting times
tfun = lambda x: datetime.strptime(str(x), '%H%M').time()
df['SA_eff'] = df['SA'].apply(tfun)
df['SU_eff'] = df['SU'].apply(tfun)
df = df.drop(columns=['SU', 'SA'])


df = df.rename(columns={'SSD' : 'SSD_eff'})
df

,SSD_eff,SA_eff,SU_eff
2020-01-01,352,09:20:00,15:12:00
2020-01-02,352,09:20:00,15:12:00
2020-01-03,353,09:20:00,15:13:00
2020-01-04,354,09:18:00,15:14:00
2020-01-05,357,09:18:00,15:15:00
...,...,...,...
2020-12-27,349,09:20:00,15:09:00
2020-12-28,348,09:20:00,15:08:00
2020-12-29,349,09:20:00,15:09:00
2020-12-30,349,09:20:00,15:09:00


In [4]:
# apply astronomic sunrise & sunset

s = sun(lat=47.264272 ,long=11.385294)

risefun = lambda x: s.sunrise(when=x)
setfun = lambda x: s.sunset(when=x)

df['SA_ast'] = df.index.map(risefun)
df['SU_ast'] = df.index.map(setfun)

# convert to correct timezone (MEZ)
def time_plus(time, timedelta):
    start = datetime(
        2020, 1, 1,
        hour=time.hour, minute=time.minute, second=time.second)
    end = start + timedelta
    return end.time()

tplu = lambda x: time_plus(x, timedelta(hours = 1))
df['SA_ast'] = df['SA_ast'].apply(tplu)
df['SU_ast'] = df['SU_ast'].apply(tplu)

# calculate astronomical sunshine duration
df['SSD_ast'] = pd.Series()
#tmp=[]
for x in range(0,len(df)):
    df['SSD_ast'][x] = (datetime.combine(df.index[x], df['SU_ast'][x]) - datetime.combine(df.index[x], df['SA_ast'][x])).total_seconds()/60
df

C:\tools\miniconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,SSD_eff,SA_eff,SU_eff,SA_ast,SU_ast,SSD_ast
2020-01-01,352,09:20:00,15:12:00,08:01:14,16:32:53,511.650000
2020-01-02,352,09:20:00,15:12:00,08:01:18,16:33:46,512.466667
2020-01-03,353,09:20:00,15:13:00,08:01:19,16:34:42,513.383333
2020-01-04,354,09:18:00,15:14:00,08:01:18,16:35:40,514.366667
2020-01-05,357,09:18:00,15:15:00,08:01:13,16:36:39,515.433333
...,...,...,...,...,...,...
2020-12-27,349,09:20:00,15:09:00,08:00:26,16:29:33,509.116667
2020-12-28,348,09:20:00,15:08:00,08:00:42,16:30:16,509.566667
2020-12-29,349,09:20:00,15:09:00,08:00:55,16:31:02,510.116667
2020-12-30,349,09:20:00,15:09:00,08:01:06,16:31:50,510.733333


In [5]:
df['SSD_ast_hhmm'] = pd.Series()
df['SSD_eff_hhmm'] = pd.Series()
for x in range(0,len(df)):
    df['SSD_ast_hhmm'].iloc[x] =  (df.index[x] + pd.Timedelta(minutes=df['SSD_ast'].iloc[x])).time()
    df['SSD_eff_hhmm'].iloc[x] =  (df.index[x] + pd.Timedelta(minutes=df['SSD_eff'].iloc[x])).time()
df

C:\tools\miniconda3\lib\site-packages\pandas\core\indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,SSD_eff,SA_eff,SU_eff,SA_ast,SU_ast,SSD_ast,SSD_ast_hhmm,SSD_eff_hhmm
2020-01-01,352,09:20:00,15:12:00,08:01:14,16:32:53,511.650000,08:31:39,05:52:00
2020-01-02,352,09:20:00,15:12:00,08:01:18,16:33:46,512.466667,08:32:28,05:52:00
2020-01-03,353,09:20:00,15:13:00,08:01:19,16:34:42,513.383333,08:33:23,05:53:00
2020-01-04,354,09:18:00,15:14:00,08:01:18,16:35:40,514.366667,08:34:22,05:54:00
2020-01-05,357,09:18:00,15:15:00,08:01:13,16:36:39,515.433333,08:35:26,05:57:00
...,...,...,...,...,...,...,...,...
2020-12-27,349,09:20:00,15:09:00,08:00:26,16:29:33,509.116667,08:29:07,05:49:00
2020-12-28,348,09:20:00,15:08:00,08:00:42,16:30:16,509.566667,08:29:34,05:48:00
2020-12-29,349,09:20:00,15:09:00,08:00:55,16:31:02,510.116667,08:30:07,05:49:00
2020-12-30,349,09:20:00,15:09:00,08:01:06,16:31:50,510.733333,08:30:44,05:49:00


In [6]:
tools_to_show = 'box_zoom,pan,save,reset,wheel_zoom'

p = figure(x_axis_type='datetime', y_axis_type='datetime', title='Effective and astronomical sunrise and sunset in Innsbruck',
              plot_width=900, plot_height=600)
p.line(x='index', y='SA_eff',source=df, color='grey', line_width=5, legend_label='Effective')
p.line(x='index', y='SU_eff',source=df, color='grey', line_width=5)
p.line(x='index', y='SA_ast',source=df, color='red', line_width=3, legend_label='Astronomical')
p.line(x='index', y='SU_ast',source=df, color='red', line_width=3)
p.line(x='index', y='SSD_ast_hhmm',source=df, color='orange', line_width=4, alpha=0.4, legend_label='Ast. sunshineduration')
p.line(x='index', y='SSD_eff_hhmm',source=df, color='green', line_width=4, alpha=0.4, legend_label='Eff. sunshineduration')
p.xaxis.formatter=DatetimeTickFormatter(
        months=["%B"],)
p.yaxis.formatter=DatetimeTickFormatter(
        hours=["%H:%M"],)
hover = HoverTool(
    tooltips = [
        ("Date", "@index{%B %d}"),
        ("Eff. Sunrise", "@SA_eff{%H:%M}"),
        ("Eff. Sunset", "@SU_eff{%H:%M}"),
        ("Ast. Sunrise", "@SA_ast{%H:%M}"),
        ("Ast. Sunset", "@SU_ast{%H:%M}"),
        ("Eff. Sunshineduration", "@SSD_eff_hhmm{%H:%M}"),
        ("Ast. Sunshineduration", "@SSD_ast_hhmm{%H:%M}"),
    ],
    formatters={
        'index': 'datetime',
        'SA_eff': 'datetime',
        'SU_eff': 'datetime',
        'SA_ast': 'datetime',
        'SU_ast': 'datetime',
        'SSD_eff_hhmm': 'datetime',
        'SSD_ast_hhmm': 'datetime',
    })
p.legend.title = 'Times in MEZ'
p.add_tools(hover)
show(p)

In [42]:

tools_to_show = 'box_zoom,pan,save,reset,wheel_zoom'
df['dummy'] = time(4)

p = figure(x_axis_type='datetime', y_axis_type='datetime', 
           title='Effective and astronomical sunrise and sunset in Innsbruck',
           plot_width=900, plot_height=600)

p1 = p.varea(x='index', y1='SSD_ast_hhmm', y2='dummy',source=df, color='yellow', alpha=0.1)
p2 = p.varea(x='index', y1='SSD_eff_hhmm', y2='dummy',source=df, color='yellow', alpha=0.2)
p3 = p.line(x='index', y='SA_eff',source=df, color='grey', line_width=5, legend_label='Effective')
p4 = p.line(x='index', y='SU_eff',source=df, color='grey', line_width=5)
p5 = p.line(x='index', y='SA_ast',source=df, color='red', line_width=3, legend_label='Astronomical')
p6 = p.line(x='index', y='SU_ast',source=df, color='red', line_width=3)
p7 = p.line(x='index', y='SSD_ast_hhmm',source=df, color='orange', line_width=4, alpha=0.2, legend_label="Sunshineduration")
p8 = p.line(x='index', y='SSD_eff_hhmm',source=df, color='orange', line_width=4, alpha=0.2)

p.xaxis.formatter=DatetimeTickFormatter(
        months=["%B"],)
p.yaxis.formatter=DatetimeTickFormatter(
        hours=["%H:%M"],)
hover = HoverTool(
    tooltips = [
        ("Date", "@index{%B %d}"),
        ("Eff. Sunrise", "@SA_eff{%H:%M}"),
        ("Eff. Sunset", "@SU_eff{%H:%M}"),
        ("Ast. Sunrise", "@SA_ast{%H:%M}"),
        ("Ast. Sunset", "@SU_ast{%H:%M}"),
        ("Eff. Sunshineduration", "@SSD_eff_hhmm{%H:%M}"),
        ("Ast. Sunshineduration", "@SSD_ast_hhmm{%H:%M}"),
    ],
    formatters={
        'index': 'datetime',
        'SA_eff': 'datetime',
        'SU_eff': 'datetime',
        'SA_ast': 'datetime',
        'SU_ast': 'datetime',
        'SSD_eff_hhmm': 'datetime',
        'SSD_ast_hhmm': 'datetime',
    },
    mode='vline')
p.legend.title = 'Times in MEZ'
hover.renderers= [p7]
p.add_tools(hover)
show(p)

In [43]:
# save to file
filename = 'innsbruck-sunshineduration.html'

html = file_html(p, CDN, "Sunshineduration in Innsbruck")

with open(filename, 'w') as f:
    f.write(html)